# Summarize the alleles for Pillar Project

In [17]:
import polars as pl

In [18]:
cc_sum_df = pl.read_csv("../../../3_outputs/1_snakemake_pipeline/3.smp_results_analyses/2.cell_count_abundance_change/2025_01_Batch_13-14/well-level_cell-count_changes.csv")
abun_sum_df = pl.read_csv("../../../3_outputs/1_snakemake_pipeline/3.smp_results_analyses/2.cell_count_abundance_change/2025_01_Batch_13-14/well-level_prot-abundance_changes.csv")
img_class_df = pl.read_csv("../../../3_outputs/1_snakemake_pipeline/3.smp_results_analyses/3.classification_analyses/2025_01_Batch_13-14/imaging_analyses_classification_summary.csv")

In [19]:
img_class_df.filter(pl.col("gene_allele").str.contains("GSS"))

gene_allele,Metadata_Bio_Batch,AUROC_BioRep1_Morph,AUROC_BioRep1_AGP,AUROC_BioRep1_GFP,AUROC_BioRep1_Mito,AUROC_BioRep1_DNA,AUROC_BioRep2_Morph,AUROC_BioRep2_AGP,AUROC_BioRep2_GFP,AUROC_BioRep2_Mito,AUROC_BioRep2_DNA,AUROC_Mean_Morph,AUROC_Mean_AGP,AUROC_Mean_GFP,AUROC_Mean_Mito,AUROC_Mean_DNA,Altered_95th_perc_Morph,Altered_95th_perc_AGP,Altered_95th_perc_GFP,Altered_95th_perc_Mito,Altered_95th_perc_DNA,Altered_99th_perc_Morph,Altered_99th_perc_AGP,Altered_99th_perc_GFP,Altered_99th_perc_Mito,Altered_99th_perc_DNA,Gene
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""GSS_Arg125Cys""","""2025_01_Batch_13-14""",0.602072,0.653072,0.763212,0.7237,0.566457,0.539237,0.619455,0.548967,0.649187,0.499309,0.570654,0.636264,0.656089,0.686444,0.532883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""GSS"""


In [20]:
img_summary_metric_df = cc_sum_df.rename(
    {
        "Variant": "gene_allele",
        "U2OS_paired_t_stat": "cc_t_stat",
        "U2OS_paired_t_pval": "cc_p_val"
    }
).select("gene_allele","cc_t_stat","cc_p_val").join(
    abun_sum_df.rename(
        {
            "Variant": "gene_allele",
            "U2OS_paired_t_stat": "abun_t_stat",
            "U2OS_paired_t_pval": "abun_p_val"
        }
    ).select("gene_allele","abun_t_stat","abun_p_val"),
    on="gene_allele",
    how="left",
    coalesce=True
).join(
    img_class_df,
    on="gene_allele",
    how="full",
    coalesce=True
).with_columns(
    pl.col("gene_allele").str.split('_').list.get(0).alias("Gene")
).filter(
    pl.col("Gene")!="CCM2"
)

In [21]:
clin_var_annot_df = pl.read_csv("../../../../../1_allele_collection/3_outputs/varchamp_alleles_with_pillar_annotations.tsv", separator="\t", infer_schema_length=10000)
clin_var_annot_df = clin_var_annot_df.with_columns(
    pl.concat_str([pl.col("symbol"), pl.col("aa_change")], separator="_").alias("gene_allele")
)

img_summary_metric_df = img_summary_metric_df.join(
    clin_var_annot_df,
    on="gene_allele",
    how="left"
)

In [24]:
# img_summary_metric_df["Interval 1 MaveDB class"]

In [25]:
img_summary_metric_df.filter(img_summary_metric_df.select("gene_variant").is_duplicated())

gene_allele,cc_t_stat,cc_p_val,abun_t_stat,abun_p_val,Metadata_Bio_Batch,AUROC_BioRep1_Morph,AUROC_BioRep1_AGP,AUROC_BioRep1_GFP,AUROC_BioRep1_Mito,AUROC_BioRep1_DNA,AUROC_BioRep2_Morph,AUROC_BioRep2_AGP,AUROC_BioRep2_GFP,AUROC_BioRep2_Mito,AUROC_BioRep2_DNA,AUROC_Mean_Morph,AUROC_Mean_AGP,AUROC_Mean_GFP,AUROC_Mean_Mito,AUROC_Mean_DNA,Altered_95th_perc_Morph,Altered_95th_perc_AGP,Altered_95th_perc_GFP,Altered_95th_perc_Mito,Altered_95th_perc_DNA,Altered_99th_perc_Morph,Altered_99th_perc_AGP,Altered_99th_perc_GFP,Altered_99th_perc_Mito,Altered_99th_perc_DNA,Gene,symbol,ensembl_gene_id,orf_id,mut_id,ccsb_mutation_id,…,clinvar_star,auth_reported_rep_score,Mode of Inheritance_ClinGen_repo,Updated_Evidence Codes_ClinGen_repo,aa_change_pp,Evidence Repo Link_ClinGen_repo,Interval 2 name,Interval 6 range,Interval 1 range,Rationale_presumed_PMID:34273903,aa_alt,clinvar_date_last_reviewed,auth_transcript_id,Interval 2 range,Naive_prior_prob_PMID:31131967,Approval Date_ClinGen_repo,simplified_consequence,Expert Panel_ClinGen_repo,aa_pos,transcript_ref,Disease_ClinGen_repo,gnomad_MAF,auth_reported_score,Interval 1 MaveDB class,clinvar_sig,hg38_start,consequence,Interval 5 MaveDB class,REVEL,auth_reported_func_class,Interval 3 name,Family History LR_PMID:31131967,PubMed Articles_ClinGen_repo,Assay_type,Interval 4 range,Calcualted_post_prob_PMID:31131967,gene_variant_pp
str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,i64,i64,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,f64,str,str,f64,f64,str,str,f64,str,str,f64,str,str,f64,str,str,str,f64,str
"""RAF1_Asp486Asn""",0.453275,1.0,-3.177824,0.025725,"""2025_01_Batch_13-14""",0.873939,0.905313,0.905897,0.929357,0.791661,0.786494,0.871518,0.885853,0.883854,0.680867,0.830216,0.888415,0.895875,0.906605,0.736264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""RAF1""","""RAF1""","""ENSG00000132155""",732,213824,"""CCSBVarC003426""",…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""RAF1_Asp486Asn""",0.453275,1.0,-3.177824,0.025725,"""2025_01_Batch_13-14""",0.873939,0.905313,0.905897,0.929357,0.791661,0.786494,0.871518,0.885853,0.883854,0.680867,0.830216,0.888415,0.895875,0.906605,0.736264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""RAF1""","""RAF1""",null,732,null,"""CCSBVarC008873""",…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""FARS2_Ile329Thr""",-2.245656,1.0,-2.124394,0.109542,"""2025_01_Batch_13-14""",0.949761,0.847965,0.792204,0.83375,0.942915,0.852707,0.741111,0.679665,0.799259,0.837768,0.901234,0.794538,0.735934,0.816504,0.890341,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,"""FARS2""","""FARS2""",null,12653,null,"""CCSBVarC011924""",…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""FARS2_Ile329Thr""",-2.245656,1.0,-2.124394,0.109542,"""2025_01_Batch_13-14""",0.949761,0.847965,0.792204,0.83375,0.942915,0.852707,0.741111,0.679665,0.799259,0.837768,0.901234,0.794538,0.735934,0.816504,0.890341,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,"""FARS2""","""FARS2""",null,null,null,"""CCSBVarC011924""",…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""SDHD_His102Arg""",4.232844,0.003902,-4.099257,0.052371,"""2025_01_Batch_13-14""",0.764633,0.830075,0.790682,0.844713,0.731416,0.693939,0.773355,0.755516,0.684436,0.687418,0.729286,0.801715,0.773099,0.764575,0.709417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""SDHD""","""SDHD""",null,6363,null,"""CCSB

In [12]:
img_summary_metric_df.filter(pl.col("gene_allele").str.contains("GSS"))

gene_allele,cc_t_stat,cc_p_val,abun_t_stat,abun_p_val,Metadata_Bio_Batch,AUROC_BioRep1_Morph,AUROC_BioRep1_AGP,AUROC_BioRep1_GFP,AUROC_BioRep1_Mito,AUROC_BioRep1_DNA,AUROC_BioRep2_Morph,AUROC_BioRep2_AGP,AUROC_BioRep2_GFP,AUROC_BioRep2_Mito,AUROC_BioRep2_DNA,AUROC_Mean_Morph,AUROC_Mean_AGP,AUROC_Mean_GFP,AUROC_Mean_Mito,AUROC_Mean_DNA,Altered_95th_perc_Morph,Altered_95th_perc_AGP,Altered_95th_perc_GFP,Altered_95th_perc_Mito,Altered_95th_perc_DNA,Altered_99th_perc_Morph,Altered_99th_perc_AGP,Altered_99th_perc_GFP,Altered_99th_perc_Mito,Altered_99th_perc_DNA,Gene,symbol,ensembl_gene_id,orf_id,mut_id,ccsb_mutation_id,…,clinvar_star,auth_reported_rep_score,Mode of Inheritance_ClinGen_repo,Updated_Evidence Codes_ClinGen_repo,aa_change_pp,Evidence Repo Link_ClinGen_repo,Interval 2 name,Interval 6 range,Interval 1 range,Rationale_presumed_PMID:34273903,aa_alt,clinvar_date_last_reviewed,auth_transcript_id,Interval 2 range,Naive_prior_prob_PMID:31131967,Approval Date_ClinGen_repo,simplified_consequence,Expert Panel_ClinGen_repo,aa_pos,transcript_ref,Disease_ClinGen_repo,gnomad_MAF,auth_reported_score,Interval 1 MaveDB class,clinvar_sig,hg38_start,consequence,Interval 5 MaveDB class,REVEL,auth_reported_func_class,Interval 3 name,Family History LR_PMID:31131967,PubMed Articles_ClinGen_repo,Assay_type,Interval 4 range,Calcualted_post_prob_PMID:31131967,gene_variant_pp
str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,i64,i64,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,f64,str,str,f64,f64,str,str,f64,str,str,f64,str,str,f64,str,str,str,f64,str
"""GSS_Arg125Cys""",4.177443,0.00415,-2.753966,0.028341,"""2025_01_Batch_13-14""",0.602072,0.653072,0.763212,0.7237,0.566457,0.539237,0.619455,0.548967,0.649187,0.499309,0.570654,0.636264,0.656089,0.686444,0.532883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""GSS""","""GSS""","""ENSG00000100983""",5132,215352,"""CCSBVarC006739""",…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [26]:
img_summary_metric_df.write_csv("../../../../../3_integrated_assay_analyses/1_inputs/imaging/imaging_analyses_summary_clinvar.tsv", separator="\t")